# Cooling Equipment Monitoring

This monitor script is used where CTs are used on circuits activated when a __room temperature exceeds a threshold value__ then deactivated when room temperature falls below the threshold value.<br>

The following issues are used in tickets opened by this script:
- CODE_RED
- CODE_ORANGE
- CODE_BLUE

### CODE_RED LOW_CURRENT_ABOVE_TCRIT<br>
This issue can arise if a circuit breaker has tripped, equipment is unplugged, drive belts are broken, or the equipment has failed open-circuit.  This is __CRITICAL__ as is is indicative of __LOW__ or __NO AIR MOVEMENT__.
### CODE_ORANGE HIGH_CURRENT_ABOVE_TCRIT<br>
This issue can arise if equipment is under excessive load caused by bearing degradation, blocked air streams, or if the equipment has failed closed-circuit.  This issue can be __CRITICAL__ or __WARNING__. 
### CODE_BLUE HIGH_CURRENT_BELOW_TCRIT<br>
This issue may occur if the switching device has failed in the __ON__ mode. This issue may be __CRITICAL__ when the room temperature is extremely low.<br>

This script currently uses fixed TCRIT and IDLEBAND settings.  Enhancements may be included at some point to calculate these parameters based on scatter diagram histories.

In [1]:
#Mote Types having the CT Sensors
CT_PROFILER_MOTE = 10003
DEPLOYED_ACTIVE  = 10001

#Ticket Types
CODE_GREEN  = 11000  # normally not ticketed
CODE_RED    = 11001
CODE_ORANGE = 11002
CODE_BLUE   = 11003

In [2]:
import os
import sys
sys.path.append('/home/sensei/jupy-notebooks/Analytics/PorterFarms')
print("============================================")
print("/  Cooling_CT is running.                  /")
print("============================================")
import requests
from datetime import datetime, timedelta
import pytz
from slackclient import SlackClient
import json
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser
import time

print(os.getcwd())
config = configparser.ConfigParser()
config.read("../../analytics_secrets.ini")

_ACTIVE_STANDBY = config['DEFAULT']['role']
if _ACTIVE_STANDBY == 'STANDBY':
    print("STANDBY")
    raise SystemExit("Stop right there!")
else:
  _SLACK_TOKEN = config['slack']['token']
  _CHIRPSTACK_USER = config['chirpstack']['user']
  _CHIRPSTACK_PASS = config['chirpstack']['password']
  _DB_HOST  = config['kanjidb']['dbhost']
  _DB_PORT  = config['kanjidb']['dbport']
  _DB_NAME  = config['kanjidb']['dbname']
  _DB_USER  = config['kanjidb']['dbuser']
  _DB_PASS  = config['kanjidb']['dbpass']
    
  _LOG_DEBUG = 0
  _LOG_INFO  = 1
  _LOG_ERROR = 2
  _LOG_LEVEL = int(config['DEFAULT']['loglevel'])
      
def logger(level, message):
    if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  print("Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
else:
  print("You are not connected to the database.")

/  Cooling_CT is running.                  /
/home/sensei/jupy-notebooks/Analytics/PorterFarms
Python version
3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
Welcome to Jupyter Notebook.  You are connected to the Kanji database!


In [3]:
def ticketIssue(conn, node_id, ticket_type, description):
  openTicket = kt.ticketExists(conn, node_id, ticket_type, [kt._OPEN_STATUS, kt._WORKING_STATUS])
  if openTicket is None:
    query = "SELECT    node.name, node.location_id, location.description, location.slackalertchannel_id, \
                       location.imageurl, customer.slacktoken, slackchannel.idslackchannel, slackchannel.channelid \
                       FROM kanji_node node JOIN kanji_location location ON location.idlocation=node.location_id \
                       JOIN kanji_slackchannel slackchannel ON location.slackalertchannel_id=slackchannel.idslackchannel \
                       JOIN kanji_customer customer ON customer.idcustomer=node.customer_id \
                       WHERE node.idnode={}".format(node_id)
    logger(_LOG_INFO, query)
    df = pd.read_sql(query, conn)    
    locationid = df['location_id'][0]
    locationdescription = df['description'][0]
    alerttextquery = "SELECT alerttext FROM kanji_tickettype WHERE idtickettype={}".format(ticket_type)
    df2 = pd.read_sql(alerttextquery, conn)
    ticketdescription = description + " " + df2['alerttext'][0]
    slackchannelid = df['idslackchannel'][0]
    slackchannelname = df['channelid'][0]
    nodename = df['name'][0]
    mentions = "@Charlie"
    locationimageurl = df['imageurl'][0]
    slacktoken = df['slacktoken'][0]
    
    logger(_LOG_DEBUG,"channel={} token={}".format(slackchannelid,slacktoken))
    
    ticketid = kt.openticket(conn, node_id, locationid, ticketdescription, 2, 3, ticket_type, slackchannelid)
    ts = kt.slackticket(nodename, locationdescription, ticketdescription, mentions, 2, 3, locationimageurl, \
                        slacktoken,slackchannelname, ticketid, 0)
    kt.updateTicket(conn, ticketid, ts)  
    logger(_LOG_INFO, "New ticket {} created for this issue.".format(ticketid))
  else:
    logger(_LOG_INFO, "There is an existing ticket {} for this issue. {}".format(openTicket['idticket'][0], openTicket['opentimestamp'][0]))

In [4]:
messagetemplate = "[\
   {\"type\": \"section\", \
		\"text\": { \
			\"type\": \"mrkdwn\", \
			\"text\": \"*<fakeLink.toUserProfiles.com|Iris / Zelda 1-1>*\\nTuesday, January 21 4:00-4:30pm\\nBuilding 2 - Havarti Cheese (3)\\n2 guests\" \
		}, \
		\"accessory\": { \
			\"type\": \"image\", \
			\"image_url\": \"https://api.slack.com/img/blocks/bkb_template_images/notifications.png\", \
			\"alt_text\": \"calendar thumbnail\" \
		} \
   } ]"

In [5]:
def postMessageToSlack(text):    
    sc = SlackClient(_SLACK_TOKEN)
    slackchannel = "infrastructure"
    response = sc.api_call("chat.postMessage", channel=slackchannel, text=text, blocks=[])
    if not 'ok' in response or not response['ok']:
      print("Error posting message to Slack channel")
      print(response)
    else:
      print("Ok posting message to Slack channel")   

In [6]:
def characterize(df):
  _ICRIT   = 2.0
  tempon   = 0.0
  tempoff  = 0.0
  temponmax    = 0.0
  tempoffmin   = 120.0
  counton  = 0
  countoff = 0
  lastamps = 0.0 
  ampson = 0.0
  ampsoff = 0.0
  for ind in df.index:
    thistemp = df['tempf'][ind]
    thisamps = df['mesh_amps'][ind]
    if (lastamps<_ICRIT) and (thisamps>=_ICRIT):
      counton += 1
      tempon  += thistemp
      ampson  += thisamps
      if thistemp>temponmax:
        temponmax=thistemp
    elif (lastamps>_ICRIT) and (thisamps<=_ICRIT):
      countoff += 1
      tempoff += thistemp
      ampsoff += thisamps
      if thistemp<tempoffmin:
        tempoffmin=thistemp
    lastamps = thisamps
  if counton>0 and countoff>0:
    dict = {'temponmax': temponmax, 
            'avgon': tempon/counton, 
            'tempoffmin': tempoffmin, 
            'avgoff': tempoff/countoff,
            'avgonamps' : ampson/counton,
            'avgoffamps': ampsoff/countoff,
            'setpoint': (tempoffmin+temponmax)/2,
            'idleband': (temponmax-tempoffmin)/2,
            'counton' : counton,
            'countoff' : countoff}
  else:
    dict = {'temponmax': 80.0, 
            'avgon': 79.0, 
            'tempoffmin': 70.0, 
            'avgoff': 71.0,
            'setpoint': 75.0,
            'idleband': 2.0,
            'avgonamps' : 10.0,
            'avgoffamps': 0.0,
            'counton' : 99,
            'countoff' : 99}
  #logger(_LOG_INFO,"Inferred Thermostat Settings:")  
  #logger(_LOG_INFO, dict)
  return dict

# Strategy
Determine if circuit current is within the polygon on the current(T) plot

Plot the polygon for reference
Plot the latest value

In [7]:
#_LOG_LEVEL = _LOG_DEBUG
_FIXED = True #False
_PERIOD_HOURS = 18

from matplotlib import pyplot as plt
from shapely.geometry.polygon import Point, LinearRing
import shapely.geometry as geometry
import shapely.ops as so
from descartes import PolygonPatch

from scipy.optimize import curve_fit

_TMIN     = 20
_TMAX     = 110

# 5 Meshes are monitored, each with it's own parameters
_TCRIT    = [78.0, 75.0, 88.0]
_TIDLEBND = [6.0,   8.0,  4.0]

_IMAX     = [20.0, 20.0, 20.0]
_IBASE    = [2.0,   2.0,  2.0]
_IBVAR    = [2.1,   2.1,  2.1] 

_INOM     = [5.0, 5.0, 5.0] 
_IVAR     = [2.0, 2.0, 2.0]

_COLUMNS  = ["cval", "dval", "eval"]
_MESHNAMES= ["3&4", "5&6", "7&8"]

#which motes are sending CT_Profiler data
motequery = "SELECT node.idnode, node.name, location.description FROM kanji_node node \
             JOIN kanji_location location ON location.idlocation=node.location_id \
             WHERE nodetype_id={} AND deploystate_id={}".format(CT_PROFILER_MOTE,DEPLOYED_ACTIVE)
logger(_LOG_DEBUG, "motequery={}".format(motequery))

motes = pd.read_sql(motequery, conn)
if motes.size>0:
 for moteind in motes.index:
  motename = motes['name'][moteind]
  moteid = motes['idnode'][moteind]
  courtesymessage = "{}\n".format(motename)
  motename = motes['name'][moteind]
  motelocation = motes['description'][moteind]
  logger(_LOG_INFO,"\nProcessing Cooling data for {} {}".format(motelocation, motename))
  for n in range(0,len(_COLUMNS)):
       
    #ctquery = "SELECT  date_trunc('minute', timestamp) AS timestamp, \
    #                   avg(bval) AS tempf, \
    #                   avg({}) AS mesh_amps \
    #                   FROM kanji_eventlog WHERE node_id={} AND fcnt>0 \
    #                   AND timestamp>NOW() - INTERVAL '{} HOURS' \
    #                   GROUP BY date_trunc('minute', timestamp) \
    #                   ORDER BY date_trunc('minute', timestamp) desc;".format(_COLUMNS[n], moteid, _PERIOD_HOURS)
    ctquery = "SELECT  timestamp, \
                       bval AS tempf, \
                       {} AS mesh_amps \
                       FROM kanji_eventlog WHERE node_id={} AND fcnt>0 \
                       AND timestamp>NOW() - INTERVAL '{} HOURS' \
                       ORDER BY timestamp desc;".format(_COLUMNS[n], moteid, _PERIOD_HOURS)
    logger(_LOG_DEBUG, ctquery)
    df = pd.read_sql(ctquery, conn)    
    
    if df.size>0:
      currenttemp = df['tempf'][0]
      currentamps = df['mesh_amps'][0]
      points = []
      for ind in df.index:
        temp = df['tempf'][ind]
        amps = df['mesh_amps'][ind]        
        points.append(Point(temp, amps)) 
      
        
      # these points define 4 non-overlapping polygons
      if _FIXED==True:
        p1  = Point(_TMIN,_IBASE[n]+_IBVAR[n])
        p8  = Point(_TMIN,_IBASE[n]-_IBVAR[n])
        p12 = Point(_TMIN,_IMAX[n])
        p2  = Point(_TCRIT[n]-_TIDLEBND[n]/2,_IBASE[n]+_IBVAR[n])
        p3  = Point(_TCRIT[n]-_TIDLEBND[n]/2,_INOM[n]+_IVAR[n])
        p11 = Point(_TCRIT[n]-_TIDLEBND[n]/2,_IMAX[n])
        p6  = Point(_TCRIT[n]+_TIDLEBND[n]/2,_INOM[n]-_IVAR[n])
        p7  = Point(_TCRIT[n]+_TIDLEBND[n]/2,_IBASE[n]-_IBVAR[n])
        p4  = Point(_TMAX,_INOM[n]+_IVAR[n])
        p5  = Point(_TMAX,_INOM[n]-_IVAR[n])
        p9  = Point(_TMAX,_IBASE[n]-_IBVAR[n])
        p10 = Point(_TMAX,_IMAX[n])
      else:
        tstat = characterize(df)
        print(_LOG_INFO,"Tstat Characterize {}".format(tstat))     
        p1  = Point(_TMIN,tstat['avgoffamps']+_IBVAR[n])
        p8  = Point(_TMIN,tstat['avgoffamps']-_IBVAR[n])
        p12 = Point(_TMIN,_IMAX[n])
        p2  = Point(tstat['setpoint']-tstat['idleband'],tstat['avgoffamps']+_IBVAR[n])
        p3  = Point(tstat['setpoint']-tstat['idleband'],tstat['avgonamps']+_IVAR[n])
        p11 = Point(tstat['setpoint']-tstat['idleband'],_IMAX[n])
        p6  = Point(tstat['setpoint']+tstat['idleband'],tstat['avgonamps']-_IVAR[n])
        p7  = Point(tstat['setpoint']+tstat['idleband'],tstat['avgoffamps']-_IBVAR[n])
        p4  = Point(_TMAX,tstat['avgonamps']+_IVAR[n])
        p5  = Point(_TMAX,tstat['avgonamps']-_IVAR[n])
        p9  = Point(_TMAX,tstat['avgoffamps']-_IBVAR[n])
        p10 = Point(_TMAX,_IMAX[n])

      nominalring  = geometry.Polygon([p1,p2,p3,p4,p5,p6,p7,p8,p1])
      x, y = nominalring.exterior.coords.xy

      CODE_BLUE_ZONE =geometry.Polygon([p12,p11,p2,p1])
      xi, yi = CODE_BLUE_ZONE.exterior.coords.xy

      CODE_RED_ZONE = geometry.Polygon([p6,p5,p9,p7])
      xii, yii = CODE_RED_ZONE.exterior.coords.xy

      CODE_ORANGE_ZONE = geometry.Polygon([p11,p10,p4,p3])
      xiii, yiii = CODE_ORANGE_ZONE.exterior.coords.xy
  
      #fig = plt.figure(1, figsize=(15,10), dpi=90)
      fig, ax = plt.subplots(1, figsize=(15,10), dpi=90)
      #ax = fig.add_subplot(111)
      ring_patch = PolygonPatch(nominalring, fc='#40d93b', ec='#40d93b', fill=True, zorder=-1)
      ax.add_patch(ring_patch)
      ring_patch = PolygonPatch(CODE_BLUE_ZONE, fc='#014efe', ec='#014efe', fill=True, zorder=-1)
      ax.add_patch(ring_patch)
      ring_patch = PolygonPatch(CODE_RED_ZONE, fc='#fe0101', ec='#fe0101', fill=True, zorder=-1)
      ax.add_patch(ring_patch)
      ring_patch = PolygonPatch(CODE_ORANGE_ZONE, fc='#fe4801', ec='#fe4801', fill=True, zorder=-1)
      ax.add_patch(ring_patch)
      #ax.plot(x, y)
      ax.set_title('Cooling Profile {} {}'.format(_COLUMNS[n], _MESHNAMES[n]))
      xrange = [_TMIN, _TMAX]
      yrange = [0, _IMAX[n]]
      ax.set_xlim(*xrange)
      ax.set_ylim(*yrange)
      ax.set_aspect(1)  
      
      cpoint = []
      currentpoint = Point(currenttemp,currentamps)
      #print(currentpoint)
      cpoint.append(Point(currenttemp, currentamps)) 
      xsc = [point.x for point in cpoint]
      ysc = [point.y for point in cpoint]
          
      # scatter diagram for this period
      xs = [point.x for point in points]
      ys = [point.y for point in points]
      plt.scatter(xs, ys, color='black')
      plt.scatter(xsc, ysc, color='white')
      
      if currentpoint.within(nominalring):
      #if nominalring.contains(currentpoint):
        logger(_LOG_INFO, "Latest {} measurement t={} A={} is nominal.".format(_MESHNAMES[n], currenttemp, currentamps))
        courtesymessage += "   {} is nominal. T={:.1f}, A={:.1f}\n".format(_MESHNAMES[n], currenttemp, currentamps)
      if currentpoint.within(CODE_BLUE_ZONE):
        logger(_LOG_INFO, "Latest {} measurement t={} A={} is CODE_BLUE".format(_MESHNAMES[n], currenttemp, currentamps))
        ticketIssue(conn, moteid, CODE_BLUE, "CODE_BLUE Fans {} Temp={} Amps={}".format(_MESHNAMES[n], currenttemp, currentamps))
        postMessageToSlack("{} is CODE_BLUE alert.".format(motelocation))
      if currentpoint.within(CODE_RED_ZONE):
        logger(_LOG_INFO, "Latest {} measurement t={} A={} is CODE_RED".format(_MESHNAMES[n], currenttemp, currentamps))
        ticketIssue(conn, moteid, CODE_RED, "CODE_RED Fans {} Temp={} Amps={}".format(_MESHNAMES[n], currenttemp, currentamps))
        postMessageToSlack("{} is CODE_RED alert.".format(motelocation))
      if currentpoint.within(CODE_ORANGE_ZONE):
        logger(_LOG_INFO, "Latest {} measurement t={} A={} is CODE_ORANGE".format(_MESHNAMES[n], currenttemp, currentamps))
        ticketIssue(conn, moteid, CODE_ORANGE, "CODE_ORANGE Fans {} Temp={} Amps={}".format(_MESHNAMES[n], currenttemp, currentamps))
        postMessageToSlack("{} is CODE_ORANGE alert.".format(motelocation))

      #logger(_LOG_INFO, "Latest measurement t={} A={}".format(currenttemp, currentamps))  
      if _LOG_LEVEL==_LOG_DEBUG:
        print(nominalring)
  #if len(courtesymessage)>0:
  #  postMessageToSlack(courtesymessage)  
            


Processing Cooling data for Breeding agMote-20015


DatabaseError: Execution failed on sql 'SELECT  timestamp,                        bval AS tempf,                        cval AS mesh_amps                        FROM kanji_eventlog WHERE node_id=20015 AND fcnt>0                        AND timestamp>NOW() - INTERVAL '18 HOURS'                        ORDER BY timestamp desc;': column "bval" does not exist
LINE 1: SELECT  timestamp,                        bval AS tempf,    ...
                                                  ^
